# Introduction

This notebook outlines how to build a recommendation system using SageMaker's Factorization Machines (FM). The main goal is to showcase how to extend FM model to predict top "X" recommendations using SageMaker's KNN and Batch Transform.

There are four parts to this notebook:

1. Building a FM Model
2. Repackaging FM Model to fit a KNN Model
3. Building a KNN model
4. Running Batch Transform for predicting top "X" items


## Part 1 - Building a FM Model using movie lens dataset

Julien Simon has written a fantastic blog about how to build a FM model using SageMaker with detailed explanation. Please see the links below for more information. In this part, I utilized his code for the most part to have continutity for performing additional steps.

Source - https://aws.amazon.com/blogs/machine-learning/build-a-movie-recommender-with-factorization-machines-on-amazon-sagemaker/

In [10]:
import sagemaker
import sagemaker.amazon.common as smac
from sagemaker import get_execution_role
from sagemaker.predictor import json_deserializer
from sagemaker.amazon.amazon_estimator import get_image_uri
import numpy as np
from scipy.sparse import lil_matrix
import pandas as pd
import boto3, io, os

### Download movie rating data from movie lens

In [11]:
#download data
!wget http://files.grouplens.org/datasets/movielens/ml-100k.zip
!unzip -o ml-100k.zip

--2019-04-24 17:17:15--  http://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.34.235
Connecting to files.grouplens.org (files.grouplens.org)|128.101.34.235|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip.1’

ml-100k.zip.1       100%[===================>]   4.70M  9.22MB/s    in 0.5s    

2019-04-24 17:17:16 (9.22 MB/s) - ‘ml-100k.zip.1’ saved [4924029/4924029]

Archive:  ml-100k.zip
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base         
  inflat

### Shuffle the data

In [12]:
%cd ml-100k
!shuf ua.base -o ua.base.shuffled

/home/ec2-user/SageMaker/ml-100k/ml-100k


### Load Training Data

In [13]:
user_movie_ratings_train = pd.read_csv('ua.base.shuffled', sep='\t', index_col=False, 
                 names=['user_id' , 'movie_id' , 'rating'])
user_movie_ratings_train.head(5)

,user_id,movie_id,rating
0,752,338,3
1,57,151,3
2,937,225,2
3,666,962,3
4,236,132,4


### Load Test Data

In [14]:
user_movie_ratings_test = pd.read_csv('ua.test', sep='\t', index_col=False, 
                 names=['user_id' , 'movie_id' , 'rating'])
user_movie_ratings_test.head(5)

,user_id,movie_id,rating
0,1,20,4
1,1,33,4
2,1,61,4
3,1,117,3
4,1,155,2


In [15]:
nb_users= user_movie_ratings_train['user_id'].max()
nb_movies=user_movie_ratings_train['movie_id'].max()
nb_features=nb_users+nb_movies
nb_ratings_test=len(user_movie_ratings_test.index)
nb_ratings_train=len(user_movie_ratings_train.index)
print " # of users: ", nb_users
print " # of movies: ", nb_movies
print " Training Count: ", nb_ratings_train
print " Test Count: ", nb_ratings_test
print " Features (# of users + # of movies): ", nb_features

 # of users:  943
 # of movies:  1682
 Training Count:  90570
 Test Count:  9430
 Features (# of users + # of movies):  2625


### FM Input

Input to FM is a one-hot encoded sparse matrix. Only ratings 4 and above are considered for the model. We will be ignoring ratings 3 and below.

In [16]:
def loadDataset(df, lines, columns):
    # Features are one-hot encoded in a sparse matrix
    X = lil_matrix((lines, columns)).astype('float32')
    # Labels are stored in a vector
    Y = []
    line=0
    for index, row in df.iterrows():
            X[line,row['user_id']-1] = 1
            X[line, nb_users+(row['movie_id']-1)] = 1
            if int(row['rating']) >= 4:
                Y.append(1)
            else:
                Y.append(0)
            line=line+1

    Y=np.array(Y).astype('float32')            
    return X,Y


X_train, Y_train = loadDataset(user_movie_ratings_train, nb_ratings_train, nb_features)
X_test, Y_test = loadDataset(user_movie_ratings_test, nb_ratings_test, nb_features)

In [17]:
print(X_train.shape)
print(Y_train.shape)
assert X_train.shape == (nb_ratings_train, nb_features)
assert Y_train.shape == (nb_ratings_train, )
zero_labels = np.count_nonzero(Y_train)
print("Training labels: %d zeros, %d ones" % (zero_labels, nb_ratings_train-zero_labels))

print(X_test.shape)
print(Y_test.shape)
assert X_test.shape  == (nb_ratings_test, nb_features)
assert Y_test.shape  == (nb_ratings_test, )
zero_labels = np.count_nonzero(Y_test)
print("Test labels: %d zeros, %d ones" % (zero_labels, nb_ratings_test-zero_labels))

(90570, 2625)
(90570,)
Training labels: 49906 zeros, 40664 ones
(9430, 2625)
(9430,)
Test labels: 5469 zeros, 3961 ones


### Convert to Protobuf format for saving to S3

In [20]:
#Change this value to your own bucket name
bucket = 'movie-lens-dataset'
prefix = 'fm'

train_key      = 'train.protobuf'
train_prefix   = '{}/{}'.format(prefix, 'train')

test_key       = 'test.protobuf'
test_prefix    = '{}/{}'.format(prefix, 'test')

output_prefix  = 's3://{}/{}/output'.format(bucket, prefix)

In [21]:
def writeDatasetToProtobuf(X, bucket, prefix, key, d_type, Y=None):
    buf = io.BytesIO()
    if d_type == "sparse":
        smac.write_spmatrix_to_sparse_tensor(buf, X, labels=Y)
    else:
        smac.write_numpy_to_dense_tensor(buf, X, labels=Y)
        
    buf.seek(0)
    obj = '{}/{}'.format(prefix, key)
    boto3.resource('s3').Bucket(bucket).Object(obj).upload_fileobj(buf)
    return 's3://{}/{}'.format(bucket,obj)
    
fm_train_data_path = writeDatasetToProtobuf(X_train, bucket, train_prefix, train_key, "sparse", Y_train)    
fm_test_data_path  = writeDatasetToProtobuf(X_test, bucket, test_prefix, test_key, "sparse", Y_test)    
  
print "Training data S3 path: ",fm_train_data_path
print "Test data S3 path: ",fm_test_data_path
print "FM model output S3 path: {}".format(output_prefix)

Training data S3 path:  s3://movie-lens-dataset/fm/train/train.protobuf
Test data S3 path:  s3://movie-lens-dataset/fm/test/test.protobuf
FM model output S3 path: s3://movie-lens-dataset/fm/output


### Run training job

You can play around with the hyper parameters until you are happy with the prediction. For this dataset and hyper parameters configuration, after 100 epochs, test accuracy was around 70% on average and the F1 score (a typical metric for a binary classifier) was around 0.74 (1 indicates a perfect classifier). Not great, but you can fine tune the model further.

In [24]:
instance_type='ml.m5.large'
fm = sagemaker.estimator.Estimator(get_image_uri(boto3.Session().region_name, "factorization-machines"),
                                   get_execution_role(), 
                                   train_instance_count=1, 
                                   train_instance_type=instance_type,
                                   output_path=output_prefix,
                                   sagemaker_session=sagemaker.Session())

fm.set_hyperparameters(feature_dim=nb_features,
                      predictor_type='binary_classifier',
                      mini_batch_size=1000,
                      num_factors=64,
                      epochs=100)

fm.fit({'train': fm_train_data_path, 'test': fm_test_data_path})

INFO:sagemaker:Creating training-job with name: factorization-machines-2019-04-24-17-19-32-928


2019-04-24 17:19:33 Starting - Starting the training job...
2019-04-24 17:19:34 Starting - Launching requested ML instances......
2019-04-24 17:20:43 Starting - Preparing the instances for training......
2019-04-24 17:21:45 Downloading - Downloading input data...
2019-04-24 17:22:25 Training - Training image download completed. Training in progress.
Docker entrypoint called with argument(s): train
[04/24/2019 17:22:26 INFO 140014762657600] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-conf.json: {u'factors_lr': u'0.0001', u'linear_init_sigma': u'0.01', u'epochs': 1, u'_wd': u'1.0', u'_num_kv_servers': u'auto', u'use_bias': u'true', u'factors_init_sigma': u'0.001', u'_log_level': u'info', u'bias_init_method': u'normal', u'linear_init_method': u'normal', u'linear_lr': u'0.001', u'factors_init_method': u'normal', u'_tuning_objective_metric': u'', u'bias_wd': u'0.01', u'use_linear': u'true', u'bias_lr': u'0.1', u'mini_batch_size': u'

[2019-04-24 17:22:42.850] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 34, "duration": 890, "num_examples": 91, "num_bytes": 5796480}
[04/24/2019 17:22:42 INFO 140014762657600] #quality_metric: host=algo-1, epoch=16, train binary_classification_accuracy <score>=0.722945054945
[04/24/2019 17:22:42 INFO 140014762657600] #quality_metric: host=algo-1, epoch=16, train binary_classification_cross_entropy <loss>=0.586402386676
[04/24/2019 17:22:42 INFO 140014762657600] #quality_metric: host=algo-1, epoch=16, train binary_f_1.000 <score>=0.767137711277
#metrics {"Metrics": {"update.time": {"count": 1, "max": 892.4551010131836, "sum": 892.4551010131836, "min": 892.4551010131836}}, "EndTime": 1556126562.850962, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "factorization-machines"}, "StartTime": 1556126561.957524}

[04/24/2019 17:22:42 INFO 140014762657600] #progress_metric: host=algo-1, completed 17 % of epochs
#metrics {"Metrics"

[2019-04-24 17:22:48.104] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 46, "duration": 970, "num_examples": 91, "num_bytes": 5796480}
[04/24/2019 17:22:48 INFO 140014762657600] #quality_metric: host=algo-1, epoch=22, train binary_classification_accuracy <score>=0.728032967033
[04/24/2019 17:22:48 INFO 140014762657600] #quality_metric: host=algo-1, epoch=22, train binary_classification_cross_entropy <loss>=0.571231604272
[04/24/2019 17:22:48 INFO 140014762657600] #quality_metric: host=algo-1, epoch=22, train binary_f_1.000 <score>=0.76805278301
#metrics {"Metrics": {"update.time": {"count": 1, "max": 972.3560810089111, "sum": 972.3560810089111, "min": 972.3560810089111}}, "EndTime": 1556126568.105624, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "factorization-machines"}, "StartTime": 1556126567.132225}

[04/24/2019 17:22:48 INFO 140014762657600] #progress_metric: host=algo-1, completed 23 % of epochs
#metrics {"Metrics":

[2019-04-24 17:23:03.616] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 82, "duration": 926, "num_examples": 91, "num_bytes": 5796480}
[04/24/2019 17:23:03 INFO 140014762657600] #quality_metric: host=algo-1, epoch=40, train binary_classification_accuracy <score>=0.735956043956
[04/24/2019 17:23:03 INFO 140014762657600] #quality_metric: host=algo-1, epoch=40, train binary_classification_cross_entropy <loss>=0.545784020057
[04/24/2019 17:23:03 INFO 140014762657600] #quality_metric: host=algo-1, epoch=40, train binary_f_1.000 <score>=0.770703311385
#metrics {"Metrics": {"update.time": {"count": 1, "max": 928.3809661865234, "sum": 928.3809661865234, "min": 928.3809661865234}}, "EndTime": 1556126583.617329, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "factorization-machines"}, "StartTime": 1556126582.688118}

[04/24/2019 17:23:03 INFO 140014762657600] #progress_metric: host=algo-1, completed 41 % of epochs
#metrics {"Metrics"

...

[2019-04-24 17:23:23.307] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 128, "duration": 814, "num_examples": 91, "num_bytes": 5796480}
[04/24/2019 17:23:23 INFO 140014762657600] #quality_metric: host=algo-1, epoch=63, train binary_classification_accuracy <score>=0.740626373626
[04/24/2019 17:23:23 INFO 140014762657600] #quality_metric: host=algo-1, epoch=63, train binary_classification_cross_entropy <loss>=0.528575992123
[04/24/2019 17:23:23 INFO 140014762657600] #quality_metric: host=algo-1, epoch=63, train binary_f_1.000 <score>=0.773067715294
#metrics {"Metrics": {"update.time": {"count": 1, "max": 817.0180320739746, "sum": 817.0180320739746, "min": 817.0180320739746}}, "EndTime": 1556126603.30871, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "factorization-machines"}, "StartTime": 1556126602.490724}

[04/24/2019 17:23:23 INFO 140014762657600] #progress_metric: host=algo-1, completed 64 % of epochs
#metrics {"Metrics"

[2019-04-24 17:23:33.651] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 152, "duration": 875, "num_examples": 91, "num_bytes": 5796480}
[04/24/2019 17:23:33 INFO 140014762657600] #quality_metric: host=algo-1, epoch=75, train binary_classification_accuracy <score>=0.746813186813
[04/24/2019 17:23:33 INFO 140014762657600] #quality_metric: host=algo-1, epoch=75, train binary_classification_cross_entropy <loss>=0.522527293782
[04/24/2019 17:23:33 INFO 140014762657600] #quality_metric: host=algo-1, epoch=75, train binary_f_1.000 <score>=0.777244953206
#metrics {"Metrics": {"update.time": {"count": 1, "max": 876.8901824951172, "sum": 876.8901824951172, "min": 876.8901824951172}}, "EndTime": 1556126613.65176, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "factorization-machines"}, "StartTime": 1556126612.773863}

[04/24/2019 17:23:33 INFO 140014762657600] #progress_metric: host=algo-1, completed 76 % of epochs
#metrics {"Metrics"

[2019-04-24 17:23:38.787] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 164, "duration": 823, "num_examples": 91, "num_bytes": 5796480}
[04/24/2019 17:23:38 INFO 140014762657600] #quality_metric: host=algo-1, epoch=81, train binary_classification_accuracy <score>=0.747692307692
[04/24/2019 17:23:38 INFO 140014762657600] #quality_metric: host=algo-1, epoch=81, train binary_classification_cross_entropy <loss>=0.519947528881
[04/24/2019 17:23:38 INFO 140014762657600] #quality_metric: host=algo-1, epoch=81, train binary_f_1.000 <score>=0.777915344734
#metrics {"Metrics": {"update.time": {"count": 1, "max": 825.408935546875, "sum": 825.408935546875, "min": 825.408935546875}}, "EndTime": 1556126618.78819, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "factorization-machines"}, "StartTime": 1556126617.961835}

[04/24/2019 17:23:38 INFO 140014762657600] #progress_metric: host=algo-1, completed 82 % of epochs
#metrics {"Metrics": {


2019-04-24 17:24:01 Uploading - Uploading generated training model
2019-04-24 17:24:01 Completed - Training job completed
[2019-04-24 17:23:49.145] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 188, "duration": 828, "num_examples": 91, "num_bytes": 5796480}
[04/24/2019 17:23:49 INFO 140014762657600] #quality_metric: host=algo-1, epoch=93, train binary_classification_accuracy <score>=0.74932967033
[04/24/2019 17:23:49 INFO 140014762657600] #quality_metric: host=algo-1, epoch=93, train binary_classification_cross_entropy <loss>=0.515392574771
[04/24/2019 17:23:49 INFO 140014762657600] #quality_metric: host=algo-1, epoch=93, train binary_f_1.000 <score>=0.779273307852
#metrics {"Metrics": {"update.time": {"count": 1, "max": 830.6639194488525, "sum": 830.6639194488525, "min": 830.6639194488525}}, "EndTime": 1556126629.146482, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "factorization-machines"}, "StartTime": 1556126628.3148

Billable seconds: 137


## Part 2 - Repackaging Model data to fit a KNN Model

Now that we have the model created and stored in SageMaker, we can download the same and repackage it to fit a KNN model.

### Download model data

In [25]:
import mxnet as mx
model_file_name = "model.tar.gz"
model_full_path = fm.output_path + "/" + fm.latest_training_job.job_name + "/output/" + model_file_name
print "Model Path: ", model_full_path

#Download FM model 
%cd ..
os.system('aws s3 cp '+model_full_path+ ' ./')

#Extract model file for loading to MXNet
os.system('tar xzvf '+model_file_name)
os.system("unzip -o model_algo-1")
os.system("mv symbol.json model-symbol.json")
os.system("mv params model-0000.params")

Model Path:  s3://movie-lens-dataset/fm/output/factorization-machines-2019-04-24-17-19-32-928/output/model.tar.gz
/home/ec2-user/SageMaker/ml-100k


0

### Extract model data to create item and user latent matrixes

In [31]:
#Extract model data
m = mx.module.Module.load('./model', 0, False, label_names=['out_label'])
V = m._arg_params['v'].asnumpy()
w = m._arg_params['w1_weight'].asnumpy()
b = m._arg_params['w0_weight'].asnumpy()

# item latent matrix - concat(V[i], w[i]).  
knn_item_matrix = np.concatenate((V[nb_users:], w[nb_users:]), axis=1)
knn_train_label = np.arange(1,nb_movies+1)

#user latent matrix - concat (V[u], 1) 
ones = np.ones(nb_users).reshape((nb_users, 1))
knn_user_matrix = np.concatenate((V[:nb_users], ones), axis=1)

In [88]:
print V.shape
print w.shape
print b.shape
print str(knn_train_label)
print knn_train_label.shape
print str(knn_item_matrix)
print knn_item_matrix.shape


(2625, 64)
(2625, 1)
(1,)
[   1    2    3 ... 1680 1681 1682]
(1682,)
[[-0.11464011  0.34701794  0.28072158 ... -0.15470599 -0.18313043
   0.05821285]
 [ 0.05240935  0.13279991  0.06855981 ...  0.14580432  0.01361193
   0.00976988]
 [ 0.05998384  0.06164078  0.02166434 ... -0.00362112  0.02645562
   0.01575344]
 ...
 [ 0.0514097  -0.04958125 -0.05086879 ... -0.05299123  0.05365661
  -0.28022096]
 [ 0.04411698 -0.04259116 -0.0430868  ...  0.04554839  0.04352051
  -0.25322652]
 [ 0.04570841  0.00243134 -0.04134593 ...  0.04733435  0.0441992
  -0.23360473]]
(1682, 65)


## Part 3 - Building KNN Model

In this section, we upload the model input data to S3, create a KNN model and save the same. Saving the model, will display the model in the model section of SageMaker. Also, it will aid in calling batch transform down the line or even deploying it as an end point for real-time inference.

This approach uses the default 'index_type' parameter for knn. It is precise but can be slow for large datasets. In such cases, you may want to use a different 'index_type' parameter leading to an approximate, yet fast answer.

In [95]:
print('KNN train features shape = ', knn_item_matrix.shape)
knn_prefix = 'knn_vit'
knn_output_prefix  = 's3://{}/{}/output'.format(bucket, knn_prefix)
knn_train_data_path = writeDatasetToProtobuf(knn_item_matrix, bucket, knn_prefix, train_key, "dense", knn_train_label)
print('uploaded KNN train data: {}'.format(knn_train_data_path))

nb_recommendations = 10

# set up the estimator
knn = sagemaker.estimator.Estimator(get_image_uri(boto3.Session().region_name, "knn"),
    get_execution_role(),
    train_instance_count=1,
    train_instance_type=instance_type,
    output_path=knn_output_prefix,
    sagemaker_session=sagemaker.Session())

knn.set_hyperparameters(feature_dim=knn_item_matrix.shape[1],
                        k=nb_recommendations,
                        index_metric="INNER_PRODUCT",
                        predictor_type='classifier',
                        sample_size=200000)
fit_input = {'train': knn_train_data_path}
knn.fit(fit_input)
knn_model_name =  knn.latest_training_job.job_name
print "created model: ", knn_model_name

# save the model so that we can reference it in the next step during batch inference
sm = boto3.client(service_name='sagemaker')
primary_container = {
    'Image': knn.image_name,
    'ModelDataUrl': knn.model_data,
}

knn_model = sm.create_model(
        ModelName = knn.latest_training_job.job_name,
        ExecutionRoleArn = knn.role,
        PrimaryContainer = primary_container)
print "saved the model"

('KNN train features shape = ', (1682, 65))
uploaded KNN train data: s3://movie-lens-dataset/knn_vit/train.protobuf


INFO:sagemaker:Creating training-job with name: knn-2019-04-24-20-52-46-443


2019-04-24 20:52:46 Starting - Starting the training job...
2019-04-24 20:52:49 Starting - Launching requested ML instances.........
2019-04-24 20:54:41 Starting - Preparing the instances for training......
2019-04-24 20:55:35 Downloading - Downloading input data...
2019-04-24 20:56:18 Training - Downloading the training image.
Docker entrypoint called with argument(s): train
[04/24/2019 20:56:22 INFO 140436055385920] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-conf.json: {u'index_metric': u'L2', u'_tuning_objective_metric': u'', u'_num_gpus': u'auto', u'_log_level': u'info', u'faiss_index_ivf_nlists': u'auto', u'epochs': u'1', u'index_type': u'faiss.Flat', u'_faiss_index_nprobe': u'5', u'_kvstore': u'dist_async', u'_num_kv_servers': u'1', u'mini_batch_size': u'5000'}
[04/24/2019 20:56:22 INFO 140436055385920] Reading provided configuration from /opt/ml/input/config/hyperparameters.json: {u'sample_size': u'200000', u'feature_di


2019-04-24 20:56:33 Uploading - Uploading generated training model
2019-04-24 20:56:33 Completed - Training job completed
Billable seconds: 59
created model:  knn-2019-04-24-20-52-46-443
saved the model


## Part 4 - Batch Transform

In this section, we will use SageMaker's batch transform option to batch predict top X for all the users.

In [96]:
#upload inference data to S3
knn_batch_data_path = writeDatasetToProtobuf(knn_user_matrix, bucket, knn_prefix, train_key, "dense")
print "Batch inference data path: ",knn_batch_data_path

# Initialize the transformer object
transformer =sagemaker.transformer.Transformer(
    base_transform_job_name="knn",
    model_name=knn_model_name,
    instance_count=1,
    instance_type=instance_type,
    output_path=knn_output_prefix,
    accept="application/jsonlines; verbose=true"
)

# Start a transform job:
transformer.transform(knn_batch_data_path, content_type='application/x-recordio-protobuf')
transformer.wait()


#Download predictions 
results_file_name = "inference_output"
inference_output_file = "knn/output/train.protobuf.out"
s3_client = boto3.client('s3')
s3_client.download_file(bucket, inference_output_file, results_file_name)
with open(results_file_name) as f:
    results = f.readlines()  

INFO:sagemaker:Creating transform job with name: knn-2019-04-24-20-56-59-717


Batch inference data path:  s3://movie-lens-dataset/knn_vit/train.protobuf
........................................!


In [97]:
# Building move lock up table.
import csv

movies = {} 
with open('u.item','r') as f:
    samples=csv.reader(f,delimiter='|')
    for movieItem in samples:
        movies[str(int(movieItem[0])-1)] = str(movieItem[1]) 

In [101]:
import json
test_user_idx = 0
u_one_json = json.loads(results[test_user_idx])

In [102]:
print "Recommended movie Ids for user #{}".format(test_user_idx+1)
i=0
for movie_id in u_one_json['labels']:
    if i == nb_recommendations:
        break
    i=i+1
    print str(i) +") "+ movies[str(int(movie_id)-1)]

Recommended movie Ids for user #1
1) Lawrence of Arabia (1962)
2) Schindler's List (1993)
3) One Flew Over the Cuckoo's Nest (1975)
4) Usual Suspects, The (1995)
5) Godfather, The (1972)
6) Star Wars (1977)
7) Silence of the Lambs, The (1991)
8) Vertigo (1958)
9) Shawshank Redemption, The (1994)
10) Casablanca (1942)


In [109]:
 print movies[str(208-1)]

Young Frankenstein (1974)


In [103]:

print "Recommended movie Ids for user #{} : {}".format(test_user_idx+1, [int(movie_id) for movie_id in u_one_json['labels']])
print
print "Movie distances for user #{} : {}".format(test_user_idx+1,  [round(distance, 4) for distance in u_one_json['distances']])

Recommended movie Ids for user #1 : [511, 318, 357, 12, 127, 50, 98, 479, 64, 483]

Movie distances for user #1 : [3.3666, 3.4212, 3.4312, 3.4713, 3.6564, 3.6715, 3.8128, 3.8352, 3.9003, 4.0307]
